In [ ]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.2/676.2 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 12.4 MB/s eta 0:00:00


In [ ]:
import mlflow
import mlflow.spark
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, randn
import random


In [ ]:
def log_synthetic_data():
    # Set MLflow tracking URI
    mlflow.set_tracking_uri("http://mlflow-server:5000")

    # Create or get experiment
    experiment_name = "synthetic_data_experiment" # Use your roll number
    try:
        experiment_id = mlflow.create_experiment(experiment_name)
    except mlflow.exceptions.MlflowException:
        experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

    mlflow.set_experiment(experiment_name)

    # Initialize Spark session
    spark = SparkSession.builder \
        .appName("MLflow Synthetic Data") \
        .getOrCreate()
        #    .config("spark.executor.memory", "512m") \
        #    .config("spark.executor.core", "2") \
        #    .config("spark.driver.memory", "2g") \
        #    .config("spark.executor.instance", "2")\
        #    .config("spark.driver.core", "1") \


    with mlflow.start_run():
        # Generate synthetic parameters
        learning_rate = random.uniform(0.001, 0.1)
        batch_size = random.choice([32, 64, 128, 256])
        epochs = random.randint(10, 100)

        # Log parameters
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("epochs", epochs)

        # Generate synthetic metrics
        accuracy = random.uniform(0.7, 0.95)
        loss = random.uniform(0.1, 0.8)
        f1_score = random.uniform(0.65, 0.92)

        # Log metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("loss", loss)
        mlflow.log_metric("f1_score", f1_score)

        # Generate synthetic Spark DataFrame
        df = spark.range(1000) \
            .withColumn("feature_1", rand(seed=42)) \
            .withColumn("feature_2", randn(seed=123)) \
            .withColumn("target", (rand() > 0.5).cast("int"))

        # Log dataset info
        mlflow.log_param("dataset_size", df.count())
        mlflow.log_param("num_features", len(df.columns) - 1)

        # Save DataFrame as artifact (parquet format)
        temp_path = "/tmp/synthetic_data"
        df.write.mode("overwrite").parquet(temp_path)
        mlflow.log_artifacts(temp_path, "synthetic_dataset")

        # Log some synthetic model artifacts
        model_summary = f"""
        Model Type: Random Forest
        Learning Rate: {learning_rate}
        Batch Size: {batch_size}
        Epochs: {epochs}
        Final Accuracy: {accuracy:.4f}
        Final Loss: {loss:.4f}
        """

        with open("model_summary.txt", "w") as f:
            f.write(model_summary)
        mlflow.log_artifact("model_summary.txt")

        print(f"Run completed successfully!")
        print(f"Experiment: {experiment_name}")
        print(f"Run ID: {mlflow.active_run().info.run_id}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Loss: {loss:.4f}")

    spark.stop()



In [ ]:
if __name__ == "__main__":
    log_synthetic_data()
